# 모델링을 위한 질문 생성 코드

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# %pip install nest_asyncio

In [4]:
import os
import asyncio
import json
import random
import nest_asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from tqdm.asyncio import tqdm

# 1. 환경 설정
nest_asyncio.apply()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.8)

INPUT_FILE = './data/FINAL_MASTER_DATA_FIXED_7757.json'
OUTPUT_FILE = './data/FINAL_PERSONA_DATASET.json'

# --- 💡 병렬 처리 및 성능 설정 ---
MAX_CONCURRENT_REQUESTS = 10  # 동시 요청 수 (OpenAI 티어에 따라 조절)
SAVE_INTERVAL = 20            # 20건마다 중간 저장
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

# 2. 프롬프트 정의 (라이프스타일 중심)
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 서비스의 사용자 경험(UX) 데이터 설계자입니다. 카드의 세부 정보를 분석하여, 실제 사용자가 모바일 앱에서 상담원에게 고민을 털어놓는 듯한 자연스러운 1인칭 질문과 그에 대한 전문적인 답변 쌍을 생성하세요."),
    ("user", """아래 [카드 정보]를 바탕으로 질문(anchor)과 정답(positive)을 생성하세요.

[카드 정보]
- 카드명: {card_name}
- 핵심 혜택: {summary}
- 주요 가맹점: {merchants}
- 상세 혜택: {benefit_detail}
- 이용 조건: {condition}
- 유의 사항: {additional_info}

[생성 가이드]
1. anchor: 위 정보 중 '주요 가맹점'이나 '핵심 혜택'을 보고, 이 서비스가 필요한 사용자의 구체적인 일상 상황이나 소비 습관을 상상하여 100~150자 내외로 질문을 작성하세요. (카드 이름은 언급하지 않습니다.)
2. positive: 사용자의 상황에 깊이 공감하며 해결책을 제시하세요. {summary}를 중심으로 답변하되, 질문자가 꼭 알아야 할 {condition}과 {additional_info}의 핵심 내용을 포함하여 150~200자 내외로 작성하세요.
3. 결과는 반드시 아래 JSON 형식으로만 답변하세요:
{{
    "anchor": "사용자의 라이프스타일이 담긴 질문",
    "positive": "공감과 상세 조건이 포함된 정답"
}}
""")
])

async def process_item(item, all_data, chain):
    """단일 아이템에 대한 AI 호출 및 결과 처리"""
    async with semaphore:
        s = item['ai_structured']
        m = item['metadata']
        
        try:
            res = await chain.ainvoke({
                "card_name": m['card_name'],
                "summary": s.get('summary', ''),
                "merchants": ", ".join(s.get('merchants', [])),
                "condition": s.get('condition', '전월 실적 무관'),
                "benefit_detail": s.get('benefit_detail', ''),
                "additional_info": s.get('additional_info', '특이사항 없음')
            })
            
            ai_data = json.loads(res.content.strip().replace('```json', '').replace('```', ''))
            
            # 오답(Negative) 구성: 메인 카테고리가 다른 데이터를 가져와 변별력 확보
            other_item = random.choice([x for x in all_data if x['metadata'].get('main_category') != m.get('main_category')])
            negative_text = other_item['ai_structured'].get('summary', '관련 혜택 정보가 없습니다.')
            
            return {
                "anchor": ai_data['anchor'],
                "positive": ai_data['positive'],
                "negative": negative_text,
                "metadata": {
                    "card_id": m.get('card_id'), # 나중에 매칭 확인용
                    "card_name": m['card_name'],
                    "corp": m['corp'],
                    "main_category": m.get('main_category'),
                    "sub_category": m.get('sub_category')
                }
            }
        except Exception as e:
            # 에러 발생 시 None 반환하여 나중에 필터링
            return None

async def run_main():
    # 1. 원본 데이터 로드
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        all_data = json.load(f)

    # 2. 중간 저장된 데이터 로드 (이어하기 로직)
    results = []
    processed_ids = set()
    if os.path.exists(OUTPUT_FILE):
        with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
            results = json.load(f)
            # card_id와 카테고리 조합으로 고유성 판단 (청크 단위이므로)
            for r in results:
                processed_ids.add(f"{r['metadata']['card_id']}_{r['metadata']['main_category']}")
        print(f"🔄 이어하기: 기존 {len(results)}건의 데이터를 불러왔습니다.")

    # 3. 아직 처리되지 않은 데이터만 선별
    to_process = [
        item for item in all_data 
        if f"{item['metadata']['card_id']}_{item['metadata']['main_category']}" not in processed_ids
    ]
    
    if not to_process:
        print("✅ 모든 데이터가 이미 처리되었습니다.")
        return

    print(f"🚀 총 {len(to_process)}건의 데이터 생성을 시작합니다... (동시 요청: {MAX_CONCURRENT_REQUESTS})")

    chain = prompt | llm
    
    # 4. 병렬 처리 루프
    # SAVE_INTERVAL 단위로 묶어서 처리
    for i in range(0, len(to_process), SAVE_INTERVAL):
        batch = to_process[i : i + SAVE_INTERVAL]
        tasks = [process_item(item, all_data, chain) for item in batch]
        
        # tqdm으로 현재 배치 진행률 표시
        batch_results = await tqdm.gather(*tasks, desc=f"Building Batch ({i//SAVE_INTERVAL + 1})")
        
        # None(에러 건) 제외하고 결과 추가
        valid_results = [r for r in batch_results if r is not None]
        results.extend(valid_results)
        
        # 중간 저장
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
            
    print(f"✨ 전체 작업 완료! 최종 파일: {OUTPUT_FILE} (총 {len(results)}건)")

# 실행
if __name__ == "__main__":
    await run_main()

🚀 총 7757건의 데이터 생성을 시작합니다... (동시 요청: 10)


Building Batch (388): 100%|██████████| 17/17 [00:06<00:00,  2.82it/s]

✨ 전체 작업 완료! 최종 파일: ./data/FINAL_PERSONA_DATASET.json (총 7757건)


In [12]:
import pandas as pd
import matplotlib.pyplot as plt

 # 한국어 형태소 분석기

# 1. 데이터 로드
with open('./data/FINAL_PERSONA_DATASET.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
# metadata 내의 정보를 컬럼으로 확장
df = pd.concat([df.drop(['metadata'], axis=1), df['metadata'].apply(pd.Series)], axis=1)

# 2. 기초 통계 및 결측치 확인
print(f"✅ 전체 데이터 건수: {len(df)}")
print(f"✅ 결측치 합계:\n{df.isnull().sum()}")
print(f"✅ 중복 데이터 건수: {df.duplicated(['anchor']).sum()}")


# 5. 카드사(corp)별 데이터 비중
print("\n🏢 카드사별 데이터 비중:")
print(df['corp'].value_counts(normalize=True) * 100)


duplicates = df[df.duplicated(['anchor'], keep=False)].sort_values(by='anchor')
print("🔍 중복된 데이터 샘플:")
display(duplicates[['anchor', 'card_name', 'main_category', 'card_id']])

✅ 전체 데이터 건수: 7757
✅ 결측치 합계:
anchor           0
positive         0
negative         0
card_id          0
card_name        0
corp             0
main_category    0
sub_category     0
dtype: int64
✅ 중복 데이터 건수: 6

🏢 카드사별 데이터 비중:
corp
KB국민카드          15.057368
신한카드            12.659533
우리카드            10.983628
NH농협카드           8.134588
삼성카드             7.812299
롯데카드             7.747841
현대카드             7.361093
하나카드             7.232177
IBK기업은행          5.182416
BC 바로카드          2.694341
MG새마을금고          2.333376
BNK부산은행          1.430966
우체국              1.379399
iM뱅크             1.082893
씨티카드             0.889519
신협               0.863736
광주은행             0.837953
제주은행             0.773495
케이뱅크             0.670362
전북은행             0.580121
SC제일은행           0.515663
Sh수협은행           0.438314
SBI저축은행          0.386747
코나카드             0.257832
교보증권             0.244940
카카오뱅크            0.206265
현대백화점            0.193374
KB증권             0.193374
한패스              0.154699
SSGPAY. CARD     

,anchor,card_name,main_category,card_id
4436,"가족과 함께 놀이공원에 자주 가는데, 할인 혜택이 있으면 좋겠어요. 어떤 카드가 도...",Oil & Life카드(Life카드),문화/레저,259
7204,"가족과 함께 놀이공원에 자주 가는데, 할인 혜택이 있으면 좋겠어요. 어떤 카드가 도...",AK우리카드 체크,문화/레저,599
6140,"온라인 쇼핑을 자주 하는데, 쿠팡이나 위메프에서 할인 혜택을 받을 수 있는 방법이 ...",일상의 기쁨카드(체크),쇼핑/생활,441
7750,"온라인 쇼핑을 자주 하는데, 쿠팡이나 위메프에서 할인 혜택을 받을 수 있는 방법이 ...",해외에선 체크카드,쇼핑/생활,689
425,"온라인 쇼핑을 자주 하는데, 할인 혜택을 최대한 활용하고 싶어요. 어떤 카드가 좋을까요?",디지로카 Paris,쇼핑/생활,2633
1853,"온라인 쇼핑을 자주 하는데, 할인 혜택을 최대한 활용하고 싶어요. 어떤 카드가 좋을까요?",디지로카 Monaco,쇼핑/생활,2634
6776,"온라인 쇼핑을 자주 하는데, 할인 혜택을 최대한 활용하고 싶어요. 어떤 카드가 좋을까요?",NC다이노스체크카드,쇼핑/생활,372
7080,"자주 GS25에서 간단한 간식이나 음료를 사는데, 할인 혜택이 있으면 좋겠어요. 어...",KB국민 티머니노리학생증 체크카드(그린_세로형),쇼핑/생활,1617
7330,"자주 GS25에서 간단한 간식이나 음료를 사는데, 할인 혜택이 있으면 좋겠어요. 어...",KB국민 티머니노리학생증 체크카드(그린),쇼핑/생활,1618
1923,"항공 여행을 자주 하는데, 마일리지를 쉽게 적립할 수 있는 카드가 필요해요. 어떤 ...",아시아나 마일리지카드,여행/프리미엄,2118


# 데이터셋 로드 및 분할 (Train,Val,Test)

In [14]:
import json
import random
import pandas as pd
from sklearn.model_selection import train_test_split

def prepare_gemma_final_dataset(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    df = pd.DataFrame(data)
    
    # 1. 중복 제거
    df = df.drop_duplicates(subset=['anchor'], keep='first').reset_index(drop=True)

    # 2. EmbeddingGemma 공식 프롬프트 적용 (이미지 기준)
    # 질문(Query): 'task: search result | query: ' 사용
    df['anchor'] = "task: search result | query: " + df['anchor']
    
    # 정답(Document): 카드 이름을 title로 활용
    df['positive'] = df.apply(lambda x: f"title: {x['metadata']['card_name']} | text: {x['positive']}", axis=1)
    
    # 오답(Document): 특정 제목이 없으므로 'none' 처리
    df['negative'] = "title: none | text: " + df['negative']

    # 3. 데이터 셔플 및 8:1:1 분할
    full_dataset = df.to_dict('records')
    random.shuffle(full_dataset)

    train_data, temp_data = train_test_split(full_dataset, test_size=0.2, random_state=42)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    # 4. 저장 (./data 폴더 기준)
    os.makedirs('./data', exist_ok=True)
    split_files = {
        'train_dataset.json': train_data,
        'val_dataset.json': val_data,
        'test_dataset.json': test_data
    }

    for filename, content in split_files.items():
        with open(f"./data/{filename}", 'w', encoding='utf-8') as f:
            json.dump(content, f, ensure_ascii=False, indent=2)
        print(f"💾 {filename} 저장 완료: {len(content)}건")

    return train_data, val_data, test_data

# 실행
train_set, val_set, test_set = prepare_gemma_final_dataset('./data/FINAL_PERSONA_DATASET.json')

💾 train_dataset.json 저장 완료: 6200건
💾 val_dataset.json 저장 완료: 775건
💾 test_dataset.json 저장 완료: 776건
